In LangChain, tool binding refers to the process of connecting a tool (a function, API call, or external capability) to an LLM so the model can decide when and how to use that tool during reasoning. When a tool is “bound,” LangChain provides the model with a structured description of the tool’s name, inputs, and outputs, enabling the LLM to call it autonomously as part of its chain-of-thought execution. This allows the model to augment its capabilities—such as performing calculations, querying databases, or fetching external data—by invoking tools as needed within an agent or runnable workflow.

In [ ]:
import os
os.environ['OPENAI_API_KEY']= "your_api_key_here"

In [ ]:
!pip install -q langchain_openai langchai-core requests

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

## Tool Creation

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b, this tool returns their product"""
  return a * b

In [ ]:
print(multiply.invoke({"a": 6, "b": 8}))

In [ ]:
@tool
def add(a: int, b: int) -> int:
  """Given 2 numbers a and b, this tool returns their sum"""
  return a + b

In [ ]:
print(add.invoke({"a": 6, "b": 8}))

## Tool Binding

In [ ]:
llm= ChatOpenAI()

llm_with_tools= llm.bind_tools([multiply, add])     # like this we can bind multiple tools here

## NOTE: Not every llm has the capacity to bind tools in them. Only some of them have.

## Tool Calling

In [ ]:
print(llm_with_tools.invoke("Hii how are how?"))      # For this query, LLM does not need to access our tool

In [ ]:
print(llm_with_tools.invoke("What's 8 multiplied by 6"))      # For this query, LLM needs to access our tool

Now in the just above query you will see that LLM only returns the specific tool (in our case it is 'multiply') to be needed to perform our multiplication, but it actually does not perform the multiplication itself using the tool. The multiplication needs to be handled either by LangChain or by you.

In [ ]:
res= llm_with_tools.invoke("What's 8 multiplied by 6")
res.tool_calls[0]     # will print the tool needed to fulfil our requiremnt

## Tool Execution
To execute he suggested tool

In [ ]:
query= HumanMessage("What's 8 multiplied by 6")

messages= [query]

print(messages)

In [ ]:
result= llm_with_tools.invoke(messages)

messages.append(result)

print(messages)

In [ ]:
result.tool_calls[0]

In [ ]:
tool_name = result.tool_calls[0]['name']

# Call your actual function
if tool_name == "multiply":
    output = multiply(result.tool_calls[0])     # Will return a ToolMessage Object
elif tool_name == "add":
    output = multiply(result.tool_calls[0])     # Will return a ToolMessage Object

print(output)

In [ ]:
messages.append(output)

In [ ]:
final_output= llm_with_tools.invoke(messages)

print(final_output.content)